In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import HuberRegressor, LogisticRegression, Ridge
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics

import utils

In [2]:
import utils

TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 
         'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

save_suffix = ""
verbose = True

In [3]:
"""0. Data reforming"""

if verbose:
    print("--- PREPARING DATASET ...")

# Load data
feat_train = pd.read_csv("./train_features.csv")
labl_train = pd.read_csv("./train_labels.csv")
feat_test = pd.read_csv("./test_features.csv")
feat_test

--- PREPARING DATASET ...


,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,0,1,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2,39.0,NaN,44.2,17.0,NaN,36.0,10.2,13.0,...,119.0,100.0,NaN,98.0,31.0,82.0,21.8,NaN,119.0,NaN
2,0,3,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,78.0,NaN,NaN,125.0,7.34
3,0,4,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,80.0,NaN,NaN,136.0,NaN
4,0,5,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,83.0,NaN,NaN,135.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151963,9997,8,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,84.0,NaN,NaN,103.0,NaN
151964,9997,9,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,83.0,NaN,NaN,110.0,NaN
151965,9997,10,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,111.0,NaN
151966,9997,11,57.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,89.0,NaN,NaN,118.0,NaN


In [4]:
# Fill NaN entries
feat_train.fillna(0.0, inplace=True)
feat_test.fillna(0.0, inplace=True)

# feat_train.fillna(method='ffill', inplace=True)
# feat_train.fillna(0.0, inplace=True)
# feat_train

# feat_test.fillna(method='ffill', inplace=True)
# feat_test.fillna(0.0, inplace=True)
# feat_test
# feat_gp = feat_train.groupby("pid", sort=False)
# for pid, feat_data in feat_gp:
#    ind = feat_data.index
#    feat_data.fillna(method='ffill', inplace=True) 
#    feat_data.fillna(0.0, inplace=True)
#    # feat_data.fillna(feat_data.mean(), inplace=True)
#    feat_train.iloc[ind,:]=feat_data
#    print(ind)

In [5]:
# feat_train.iloc[:,3:] = (feat_train.iloc[:,3:]-feat_train.iloc[:,3:].min())/(feat_train.iloc[:,3:].max()-feat_train.iloc[:,3:].min())
feat_train.iloc[:,3:] = (feat_train.iloc[:,3:]-feat_train.iloc[:,3:].mean())/feat_train.iloc[:,3:].std()
feat_train

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,-0.205798,-0.181176,1.122617,-0.167903,1.296527,2.376658,4.160038,...,-0.099083,0.476461,-0.025058,4.218793,2.014597,0.614085,-0.069164,-0.035736,0.788594,2.829872
1,1,4,34.0,-0.205798,-0.181176,-0.230703,-0.167903,1.296527,-0.322522,-0.237106,...,-0.099083,0.476461,-0.025058,-0.254581,-0.361917,0.769230,-0.069164,-0.035736,0.445836,2.829872
2,1,5,34.0,-0.205798,-0.181176,-0.230703,-0.167903,1.296527,-0.322522,-0.237106,...,-0.099083,0.476461,-0.025058,-0.254581,-0.361917,0.552026,-0.069164,-0.035736,0.143403,2.847232
3,1,6,34.0,-0.205798,-0.181176,-0.230703,-0.167903,1.353179,-0.322522,-0.237106,...,-0.099083,0.476461,-0.025058,-0.254581,-0.361917,0.427910,-0.069164,-0.035736,0.022430,2.847232
4,1,7,34.0,-0.205798,-0.181176,-0.230703,-0.167903,-0.742950,-0.322522,-0.237106,...,-0.099083,0.476461,-0.025058,-0.254581,1.802063,0.210706,-0.069164,-0.035736,-0.058219,2.864592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,8,85.0,-0.205798,-0.181176,-0.230703,-0.167903,-0.742950,-0.322522,-0.237106,...,-0.099083,-2.434507,-0.025058,-0.254581,-0.361917,0.179677,-0.069164,-0.035736,0.143403,-0.351325
227936,9999,9,85.0,-0.205798,-0.181176,-0.230703,-0.167903,-0.742950,-0.322522,-0.237106,...,-0.099083,-2.434507,-0.025058,-0.254581,-0.361917,0.272764,-0.069164,-0.035736,0.405512,-0.351325
227937,9999,10,85.0,-0.205798,-0.181176,-0.230703,-0.167903,1.296527,-0.322522,-0.237106,...,-0.099083,0.418242,-0.025058,-0.254581,-0.361917,0.179677,-0.069164,-0.035736,0.707945,-0.351325
227938,9999,11,85.0,-0.205798,-0.181176,-0.230703,-0.167903,-0.742950,2.842033,-0.237106,...,-0.099083,0.418242,-0.025058,-0.254581,2.632877,0.024531,-0.069164,-0.035736,0.445836,-0.351325


In [6]:
# scaler = MinMaxScaler()
# scaler.fit(feat_test)
# feat_test = scaler.transform(feat_test)
# feat_test

# feat_test.iloc[:,3:] = (feat_test.iloc[:,3:]-feat_test.iloc[:,3:].min())/(feat_test.iloc[:,3:].max()-feat_test.iloc[:,3:].min())
feat_test.iloc[:,3:] = (feat_test.iloc[:,3:]-feat_test.iloc[:,3:].mean())/feat_test.iloc[:,3:].std()
feat_test

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,0,1,39.0,-0.205993,-0.182451,-0.232358,-0.170456,-0.743754,-0.322737,-0.238095,...,-0.106497,-2.440515,-0.025744,-0.256599,-0.362544,-2.312802,-0.069245,-0.037149,-2.101197,-0.352598
1,0,2,39.0,-0.205993,4.295678,1.672104,-0.170456,1.294997,2.834180,2.139149,...,4.927343,0.475423,-0.025744,3.560195,2.620623,0.240049,38.620160,-0.037149,0.322537,-0.352598
2,0,3,39.0,-0.205993,-0.182451,-0.232358,-0.170456,-0.743754,-0.322737,-0.238095,...,-0.106497,0.475423,-0.025744,-0.256599,-0.362544,0.115519,-0.069245,-0.037149,0.444742,2.823117
3,0,4,39.0,-0.205993,-0.182451,-0.232358,-0.170456,-0.743754,-0.322737,-0.238095,...,-0.106497,0.475423,-0.025744,-0.256599,-0.362544,0.177784,-0.069245,-0.037149,0.668785,-0.352598
4,0,5,39.0,-0.205993,-0.182451,-0.232358,-0.170456,-0.743754,-0.322737,-0.238095,...,-0.106497,0.475423,-0.025744,-0.256599,-0.362544,0.271181,-0.069245,-0.037149,0.648418,-0.352598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151963,9997,8,57.0,-0.205993,-0.182451,-0.232358,-0.170456,-0.743754,-0.322737,-0.238095,...,-0.106497,0.475423,-0.025744,-0.256599,-0.362544,0.302313,-0.069245,-0.037149,-0.003343,-0.352598
151964,9997,9,57.0,-0.205993,-0.182451,-0.232358,-0.170456,-0.743754,-0.322737,-0.238095,...,-0.106497,0.475423,-0.025744,-0.256599,-0.362544,0.271181,-0.069245,-0.037149,0.139230,-0.352598
151965,9997,10,57.0,-0.205993,-0.182451,-0.232358,-0.170456,-0.743754,-0.322737,-0.238095,...,-0.106497,0.475423,-0.025744,-0.256599,-0.362544,0.426843,-0.069245,-0.037149,0.159597,-0.352598
151966,9997,11,57.0,-0.205993,-0.182451,-0.232358,-0.170456,1.351629,-0.322737,-0.238095,...,-0.106497,0.475423,-0.025744,-0.256599,-0.362544,0.457975,-0.069245,-0.037149,0.302170,-0.352598


In [7]:
# Flatten to patient feature vector and sort by pid
feat_train = utils.patient_feat_flatten(feat_train).sort_index()
# feat_test = utils.patient_feat_flatten(feat_test).sort_index()
labl_train.sort_values("pid", inplace=True)

# Initialize prediction dataframes
labl_pred_train = pd.DataFrame(index=feat_train.index, columns=labl_train.columns)
labl_pred_train.pid = feat_train.index
# labl_pred_test = pd.DataFrame(index=feat_test.index, columns=labl_train.columns)
# labl_pred_test.pid = feat_test.index


In [11]:
"""1. Subtask - test prediction (binary classification)"""

for test_labl in TESTS:
        
    # Train with complete dataset, using Logistic Regression
    test_classifier = LogisticRegression(penalty="l2", C=0.5, fit_intercept=True, 
                                           solver="newton-cg", max_iter=100) # sag, lbfgs
    test_classifier.fit(feat_train, labl_train[test_labl]) 
    
    # test_classifier = SVC(gamma=2, C=1)
    # test_classifier.fit(feat_train, labl_train[test_labl])
        
    # Recall on training set + test set
    labl_pred_train[test_labl] = test_classifier.predict_proba(feat_train)[:, 1]
    # labl_pred_test[test_labl] = test_classifier.predict_proba(feat_test)[:, 1]
    
    # labl_pred_train[test_labl] = test_classifier.decision_function(feat_train)
    # labl_pred_test[test_labl] = test_classifier.decision_function(feat_test)

In [12]:
metrics.roc_auc_score(labl_train.iloc[:, 1], labl_pred_train.iloc[:, 1]) 

0.9147836466104482

In [10]:
feat_train

Measure     EtCO2                                                              \
Time           0         1         2         3         4         5         6    
1       -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
2       -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
4       -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
6       -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
8       -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
...           ...       ...       ...       ...       ...       ...       ...   
31653   -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
31654   -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
31656   -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
31657   -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   
31658   -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798 -0.205798   

Measure                                ...        pH                      \
Time           7         8         9   ...        3         4         5    
1       -0.205798 -0.205798 -0.205798  ...  2.847232  2.864592 -0.351325   
2       -0.205798 -0.205798 -0.205798  ... -0.351325 -0.351325 -0.351325   
4       -0.205798 -0.205798 -0.205798  ... -0.351325 -0.351325 -0.351325   
6       -0.205798 -0.205798 -0.205798  ...  2.829872  2.838552  2.834212   
8       -0.205798 -0.205798 -0.205798  ... -0.351325 -0.351325 -0.351325   
...           ...       ...       ...  ...       ...       ...       ...   
31653   -0.205798 -0.205798 -0.205798  ...  2.829872  2.829872 -0.351325   
31654   -0.205798 -0.205798 -0.205798  ... -0.351325 -0.351325 -0.351325   
31656   -0.205798 -0.205798 -0.205798  ...  2.808173 -0.351325  2.834212   
31657   -0.205798 -0.205798 -0.205798  ... -0.351325 -0.351325 -0.351325   
31658   -0.205798 -0.205798 -0.205798  ... -0.351325 -0.351325 -0.351325   

Measure                                                               Age  
Time           6         7         8         9         10        11    0   
1       -0.351325 -0.351325 -0.351325  2.855912  2.855912 -0.351325  34.0  
2       -0.351325 -0.351325 -0.351325 -0.351325 -0.351325 -0.351325  86.0  
4       -0.351325 -0.351325 -0.351325 -0.351325 -0.351325 -0.351325  66.0  
6        2.855912  2.847232  2.834212 -0.351325 -0.351325 -0.351325  66.0  
8       -0.351325 -0.351325 -0.351325 -0.351325 -0.351325 -0.351325  42.0  
...           ...       ...       ...       ...       ...       ...   ...  
31653   -0.351325 -0.351325 -0.351325 -0.351325 -0.351325 -0.351325  52.0  
31654   -0.351325 -0.351325 -0.351325 -0.351325 -0.351325 -0.351325  66.0  
31656   -0.351325  2.829872 -0.351325 -0.351325 -0.351325 -0.351325  44.0  
31657   -0.351325 -0.351325 -0.351325 -0.351325 -0.351325 -0.351325  70.0  
31658   -0.351325 -0.351325 -0.351325 -0.351325 -0.351325 -0.351325  60.0  

[18995 rows x 409 columns]

In [ ]:
"""1. Subtask - test prediction (binary classification)"""
C_list = np.arange(0.1, 2, 0.2)
res = []
for C_val in C_list:
    for test_labl in TESTS:

        # Train with complete dataset, using Logistic Regression
        test_classifier = LogisticRegression(penalty="l2", C=C_val, fit_intercept=True, 
                                               solver="newton-cg", max_iter=100) # sag, lbfgs
        test_classifier.fit(feat_train, labl_train[test_labl]) 

        # test_classifier = SVC(gamma=2, C=1)
        # test_classifier.fit(feat_train, labl_train[test_labl])

        # Recall on training set + test set
        labl_pred_train[test_labl] = test_classifier.predict_proba(feat_train)[:, 1]
        # labl_pred_test[test_labl] = test_classifier.predict_proba(feat_test)[:, 1]

        # labl_pred_train[test_labl] = test_classifier.decision_function(feat_train)
        # labl_pred_test[test_labl] = test_classifier.decision_function(feat_test)
    res.append(metrics.roc_auc_score(labl_train.iloc[:, 1], labl_pred_train.iloc[:, 1]))

In [15]:
C_list = np.arange(0.1, 2, 0.2)
C_list

array([0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9])